In [9]:
import tensorflow as tf
import numpy as np
import dataset
import pickle
from dataset import DataSet
import os
from collections import Counter
import pandas as pd
import santander_constants as constants
import santander_utils as utils

# Load the model

In [10]:
# Specify model type and parameters
model_type = constants.ModelType.RNN_ATTENTION_EMBEDDINGS
parameters = {}
parameters['seq_length'] = constants.MAX_SEQ_LENGTH
parameters['n_input'] = constants.N_INPUT
parameters['n_output'] = constants.N_OUTPUT
parameters['n_hidden'] = 100
parameters['weight_init'] =  constants.WEIGHT_INITIALIZATION_DICT['normal'](0.1)
parameters['learning_rate'] = 0.001
parameters['optimizer'] = constants.OptimizerType.ADAM
parameters['rnn_type'] = constants.RnnCellType.LSTM2
parameters['num_epochs'] = 10000
parameters['batch_size'] = 1
parameters['embedding_size'] = 100              # Only for attention to embeddings model (will be ignored for others)
parameters['embedding_activation'] = 'linear'  # Only for attention to embeddings model (will be ignored for others)
parameters['attention_activation'] = constants.AttentionActivation.LINEAR
parameters['l2'] = 0.
parameters['dropout'] = 0.
parameters['layer_norm'] = False
print('RNN Type: ' + str(model_type))
print(parameters)

model = utils.create_tensorflow_graph(model_type, parameters)

checkpoint_path = 'C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712'

RNN Type: ModelType.RNN_ATTENTION_EMBEDDINGS
{'learning_rate': 0.001, 'num_epochs': 10000, 'embedding_size': 100, 'dropout': 0.0, 'optimizer': <OptimizerType.ADAM: 2>, 'weight_init': <tensorflow.python.ops.init_ops.RandomNormal object at 0x000000000D037B00>, 'embedding_activation': 'linear', 'layer_norm': False, 'n_hidden': 100, 'batch_size': 1, 'rnn_type': <RnnCellType.LSTM2: 2>, 'attention_activation': <AttentionActivation.LINEAR: 1>, 'seq_length': 18, 'l2': 0.0, 'n_output': 24, 'n_input': 48}
Variable w_alphas:0 will be regularized
Variable w_out:0 will be regularized
Variable w_emb:0 will be regularized
Variable rnn/lstm_cell/kernel:0 will be regularized
INFO:tensorflow:Summary name w_alphas:0 is illegal; using w_alphas_0 instead.
INFO:tensorflow:Summary name w_out:0 is illegal; using w_out_0 instead.
INFO:tensorflow:Summary name w_emb:0 is illegal; using w_emb_0 instead.
INFO:tensorflow:Summary name b_out:0 is illegal; using b_out_0 instead.
INFO:tensorflow:Summary name b_alphas:0

# Load Preprocessed Dataset (Pickle)

In [11]:
with open("preprocessed/dataset_augmented_07.pickle", 'rb') as handle:
    dataset = pickle.load(handle)
#ds = DataSet(dataset)
#del dataset

# Define functions

In [233]:
product_columns = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
                  'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
                  'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
                  'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
                  'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
                  'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

product_descriptions = ['Saving Account', 'Guarantees', 'Current Accounts', 'Derivada Account',
                  'Payroll Account', 'Junior Account', 'Más particular Account', 'particular Account',
                  'particular Plus Account', 'Short-term deposits', 'Medium-term deposits', 'Long-term deposits',
                  'e-account', 'Funds', 'Mortgage', 'Pensions',
                  'Loans', 'Taxes', 'Credit Card', 'Securities',
                  'Home Account', 'Payroll', 'Pensions', 'Direct Debit']


def show_predictions(model, checkpoint_path, dataset, idx_sample, set_name):

    saver = tf.train.Saver()
    print('Index Sample: ' + str(idx_sample))
    print('ncodpers: ' + str(dataset[set_name]['ncodpers'][idx_sample]))
    with tf.Session() as sess:
        saver.restore(sess, checkpoint_path)
        batch_x_test = np.expand_dims(dataset[set_name]['X'][idx_sample].toarray(), axis=0)
        predictions_test, attention_weights = sess.run([model.pred_prob, model.alphas ], 
                                                       feed_dict={model.x: batch_x_test, model.dropout_keep_prob: 1})
        timesteps = np.where(batch_x_test[0] == 1)[0][-1]
        for t in range(timesteps + 1):
            str_t = str(t) + ': '
            interactions_t = np.where(batch_x_test[0][t] == 1)[0]
            for interaction_idx in interactions_t:
                if interaction_idx > 23:
                    str_t += 'Removed ' + product_descriptions[interaction_idx - 24] + ' (' + product_columns[interaction_idx - 24] + ') '
                else:
                    str_t += 'Added ' + product_descriptions[interaction_idx] + ' (' + product_columns[interaction_idx - 24] + ') '
            str_t += ' -- Attention weight: ' + str(attention_weights[0][t])
            print(str_t)
        print('Top 7 Predictions: ')
        idx_predictions = np.arange(len(product_descriptions))
        sorted_pred, sorted_idx = zip(*sorted(zip(predictions_test[0], idx_predictions), reverse=True))
        for k in range(7):
            print(product_descriptions[sorted_idx[k]] + ' (' + product_columns[sorted_idx[k]] + ')' + ': ' + str(predictions_test[0][sorted_idx[k]]))
        print('Labels: ')
        labels_test = np.where(dataset[set_name]['Y'][idx_sample].toarray() == 1)[1]
        for lbl in labels_test:
            print(product_descriptions[lbl] + ' (' + product_columns[lbl] + ')')
    
def search_correct_predictions(model, checkpoint_path, dataset, set_name, samples_to_look, k=5, exclude_most_att_last=False):
    '''
    samples_to_look: index of the samples to look
    k: Look at top k
    exclude_most_att_last: If True, it excludes the samples that have the most attention to last intearction
    '''
    
    saver = tf.train.Saver()
    correct_predictions = []
    with tf.Session() as sess:
        saver.restore(sess, checkpoint_path)
        for idx_sample in samples_to_look:
            batch_x_test = np.expand_dims(dataset[set_name]['X'][idx_sample].toarray(), axis=0)
            predictions_test, attention_weights = sess.run([model.pred_prob, model.alphas], 
                                                           feed_dict={model.x: batch_x_test, model.dropout_keep_prob: 1})
            idx_predictions = np.arange(len(product_descriptions))
            sorted_pred, sorted_idx = zip(*sorted(zip(predictions_test[0], idx_predictions), reverse=True))
            if idx_product in sorted_idx[0:k]:
                if exclude_most_att_last:
                    num_interactions = max(np.where(batch_x_test == 1)[1])
                    if np.argmax(attention_weights[0]) < num_interactions:
                        correct_predictions.append(idx_sample)
                else:
                    correct_predictions.append(idx_sample)

    print('Samples found ' + str(len(correct_predictions)))
    return correct_predictions
    
def search_samples_with_label(dataset, set_name, label, exclude_interactions, include_interactions, exclude_same=True):
    interesting_samples = []
    for i in range(len(dataset[set_name]['X'])):
        # Check that the product we are looking for is in the labels
        if label in np.where(dataset[set_name]['Y'][i].toarray() != 0)[1]:
            times, x_int = np.where(dataset[set_name]['X'][i].toarray() != 0)
            # Check that there is at least more than 1 interaction
            if len(np.unique(times)) > 1:
                # Examples removing the target product we are looking are not really interesting, so we will exclude them
                if (exclude_same and not(label + 24) in x_int) or (not exclude_same):
                    # Check that there is not any of the interactions that we want to exclude in the previous interactions
                    if not any(x in x_int for x in exclude_interactions):
                        # Check that there is all the interactions that we want to include in the previous itneractions
                        if all(x in x_int for x in include_interactions):
                            interesting_samples.append(i)
    print('Num samples found: ' + str(len(interesting_samples)))       
    return interesting_samples

# Examples ith ind_dela_fin_ult1  (Long-term deposit)

In [100]:
idx_product = product_columns.index('ind_dela_fin_ult1')
print(idx_product)

11


In [101]:
samples_to_look = search_samples_with_label(dataset, 'train', idx_product, [], [])

Num samples found: 
1158


In [107]:
correct_samples = search_correct_predictions(model, checkpoint_path, dataset, 'train', samples_to_look, k=2, exclude_most_att_last=True)

INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
Samples found 23


In [113]:
show_predictions(model, checkpoint_path, dataset, correct_samples[5], 'train')

Index Sample: 195712
ncodpers: 1378676
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Short-term deposits (ind_deco_fin_ult1)  -- Attention weight: 0.758065
1: Added Direct Debit (ind_recibo_ult1)  -- Attention weight: 0.241935
Top 7 Predictions: 
e-account: 0.332232
Long-term deposits: 0.212454
Payroll Account: 0.0530766
Direct Debit: 0.0476052
Pensions: 0.0473728
Payroll: 0.0463691
Current Accounts: 0.0452593
Labels: 
Long-term deposits 


In [116]:
show_predictions(model, checkpoint_path, dataset, correct_samples[6], 'train')
show_predictions(model, checkpoint_path, dataset, correct_samples[7], 'train')

Index Sample: 197597
ncodpers: 1384421
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Más particular Account (ind_ctma_fin_ult1) Added Short-term deposits (ind_deco_fin_ult1)  -- Attention weight: 0.808842
1: Added Direct Debit (ind_recibo_ult1)  -- Attention weight: 0.191158
Top 7 Predictions: 
e-account: 0.95489
Long-term deposits: 0.237973
Direct Debit: 0.0239598
Payroll Account: 0.023037
Payroll: 0.0164017
Pensions: 0.0156204
Credit Card: 0.0123099
Labels: 
Long-term deposits 
e-account 
Payroll 
Pensions 
Index Sample: 198263
ncodpers: 1386461
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Más particular Account (ind_ctma_fin_ult1) Added Short-term deposits (ind_deco_fin_ult1) Removed Current Accounts (ind_cco_fin_ult1)  -- Attention weight: 0.882317
1: Added Current Accou

TAKE A LOO AT THIS TWO, IT MIGHT BE INTERESTING WITH E-ACCOUNT, PERHAPS IS CORRELATED WITH CTMA AND DECO? (THE PROPENSITY IS VERY STRONG) 

In [117]:
show_predictions(model, checkpoint_path, dataset, correct_samples[8], 'train')

Index Sample: 200020
ncodpers: 1391999
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Current Accounts (ind_cco_fin_ult1) Removed Short-term deposits (ind_deco_fin_ult1)  -- Attention weight: 0.59738
1: Added e-account (ind_ecue_fin_ult1) Removed Más particular Account (ind_ctma_fin_ult1)  -- Attention weight: 0.40262
Top 7 Predictions: 
Direct Debit: 0.420905
Long-term deposits: 0.190873
Pensions: 0.0713717
Payroll: 0.0648922
e-account: 0.0548313
Payroll Account: 0.0421798
Más particular Account: 0.0204641
Labels: 
Long-term deposits 


In [136]:
show_predictions(model, checkpoint_path, dataset, correct_samples[9], 'train')

Index Sample: 200530
ncodpers: 1393642
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Short-term deposits (ind_deco_fin_ult1)  -- Attention weight: 0.482154
1: Added Direct Debit (ind_recibo_ult1)  -- Attention weight: 0.153879
2: Removed Short-term deposits (ind_deco_fin_ult1)  -- Attention weight: 0.363968
Top 7 Predictions: 
e-account (ind_ecue_fin_ult1): 0.34621
Long-term deposits (ind_dela_fin_ult1): 0.216908
Direct Debit (ind_recibo_ult1): 0.0883549
Payroll Account (ind_cno_fin_ult1): 0.0613649
Current Accounts (ind_cco_fin_ult1): 0.0489631
Payroll (ind_nomina_ult1): 0.0421209
Pensions (ind_nom_pens_ult1): 0.0416749
Labels: 
Long-term deposits (ind_dela_fin_ult1)


THIS IS PERFECT!!

ncodpers 1445133 could also be good

# Examples with ind_ecue_fin_ult1 (e-account)

In [137]:
idx_product = product_columns.index('ind_ecue_fin_ult1')
print(idx_product)
samples_to_look = search_samples_with_label(dataset, 'train', idx_product, [], [])
correct_samples = search_correct_predictions(model, checkpoint_path, dataset, 'train', samples_to_look, k=2, exclude_most_att_last=True)

12
Num samples found: 
3959
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
Samples found 705


In [139]:
show_predictions(model, checkpoint_path, dataset, correct_samples[1], 'train')

Index Sample: 1130
ncodpers: 20340
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Más particular Account (ind_ctma_fin_ult1)  -- Attention weight: 0.905222
1: Added Direct Debit (ind_recibo_ult1)  -- Attention weight: 0.0947775
Top 7 Predictions: 
e-account (ind_ecue_fin_ult1): 0.835141
Direct Debit (ind_recibo_ult1): 0.0570995
Payroll Account (ind_cno_fin_ult1): 0.0535076
Credit Card (ind_tjcr_fin_ult1): 0.0478709
Long-term deposits (ind_dela_fin_ult1): 0.0464894
Payroll (ind_nomina_ult1): 0.0464591
Pensions (ind_nom_pens_ult1): 0.044312
Labels: 
e-account (ind_ecue_fin_ult1)


GOOD EXAMPLE:
- Added ctma which seems to be correlated with ecue
- Added Direct Debt (Irrelevant for this sample)

The highest propensity with difference is e-account, which is the correct label

In [150]:
show_predictions(model, checkpoint_path, dataset, correct_samples[12], 'train')

Index Sample: 8192
ncodpers: 59525
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Current Accounts (ind_cco_fin_ult1)  -- Attention weight: 0.0104221
1: Added Más particular Account (ind_ctma_fin_ult1) Removed Current Accounts (ind_cco_fin_ult1)  -- Attention weight: 0.871522
2: Added Current Accounts (ind_cco_fin_ult1)  -- Attention weight: 0.118056
Top 7 Predictions: 
e-account (ind_ecue_fin_ult1): 0.609257
Current Accounts (ind_cco_fin_ult1): 0.136746
Direct Debit (ind_recibo_ult1): 0.0274806
Long-term deposits (ind_dela_fin_ult1): 0.0118375
Payroll Account (ind_cno_fin_ult1): 0.00950758
Payroll (ind_nomina_ult1): 0.00943207
Pensions (ind_nom_pens_ult1): 0.00895273
Labels: 
e-account (ind_ecue_fin_ult1)


GOOD EXAMPLE (SIMILAR TO PREVIOUS):
- Added cco (Not relevant, few attention)
- Added cco (Relevant, a lot of attention)
- Added cco (Not relevant, few attention)

The highest propensity is e-account, which is the correct label

In [152]:
show_predictions(model, checkpoint_path, dataset, correct_samples[14], 'train')

Index Sample: 8621
ncodpers: 61600
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Direct Debit (ind_recibo_ult1)  -- Attention weight: 0.76689
1: Added Payroll Account (ind_cno_fin_ult1) Added Payroll (ind_nomina_ult1) Added Pensions (ind_nom_pens_ult1) Removed Current Accounts (ind_cco_fin_ult1)  -- Attention weight: 0.23311
Top 7 Predictions: 
Credit Card (ind_tjcr_fin_ult1): 0.172326
e-account (ind_ecue_fin_ult1): 0.0978559
Taxes (ind_reca_fin_ult1): 0.0830436
Current Accounts (ind_cco_fin_ult1): 0.0816585
Payroll (ind_nomina_ult1): 0.0513394
Pensions (ind_nom_pens_ult1): 0.0493762
Payroll Account (ind_cno_fin_ult1): 0.0310406
Labels: 
e-account (ind_ecue_fin_ult1)
Taxes (ind_reca_fin_ult1)


Not really for cmta, but it might be worthy to take a look at this example, and see why Taxes is in the top 3 predictions

In [157]:
show_predictions(model, checkpoint_path, dataset, correct_samples[19], 'train')

Index Sample: 13297
ncodpers: 83769
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Payroll Account (ind_cno_fin_ult1) Added Payroll (ind_nomina_ult1) Added Pensions (ind_nom_pens_ult1) Added Direct Debit (ind_recibo_ult1)  -- Attention weight: 0.0975241
1: Added Más particular Account (ind_ctma_fin_ult1)  -- Attention weight: 0.866764
2: Added Credit Card (ind_tjcr_fin_ult1)  -- Attention weight: 0.0357119
Top 7 Predictions: 
e-account (ind_ecue_fin_ult1): 0.821194
Direct Debit (ind_recibo_ult1): 0.0823378
Credit Card (ind_tjcr_fin_ult1): 0.0503651
Long-term deposits (ind_dela_fin_ult1): 0.0423084
Payroll Account (ind_cno_fin_ult1): 0.0264932
Payroll (ind_nomina_ult1): 0.0252972
Pensions (ind_nom_pens_ult1): 0.0230181
Labels: 
e-account (ind_ecue_fin_ult1)


ANOTHER GOOD EXAMPLE:
- Added cno, nomina, nom_pens, recibo
- Added ctma
- Added tjcr

Top prediction is e-account (correct)

# Examples with Funds (ind_fond_fin_ult1)

In [209]:
idx_product = product_columns.index('ind_fond_fin_ult1')
print(idx_product)
samples_to_look = search_samples_with_label(dataset, 'train', idx_product, [], [], exclude_same=True)
correct_samples = search_correct_predictions(model, checkpoint_path, dataset, 'train', samples_to_look, k=6, exclude_most_att_last=True)

13
Num samples found: 
417
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
Samples found 4


In [211]:
show_predictions(model, checkpoint_path, dataset, correct_samples[1], 'train')

Index Sample: 61490
ncodpers: 364476
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Medium-term deposits (ind_deme_fin_ult1)  -- Attention weight: 0.0915063
1: Removed Long-term deposits (ind_dela_fin_ult1)  -- Attention weight: 0.519277
2: Removed Medium-term deposits (ind_deme_fin_ult1)  -- Attention weight: 0.389217
Top 7 Predictions: 
Long-term deposits (ind_dela_fin_ult1): 0.311756
Direct Debit (ind_recibo_ult1): 0.137346
Credit Card (ind_tjcr_fin_ult1): 0.114257
e-account (ind_ecue_fin_ult1): 0.112159
Current Accounts (ind_cco_fin_ult1): 0.0672684
Funds (ind_fond_fin_ult1): 0.0483811
Pensions (ind_nom_pens_ult1): 0.03889
Labels: 
Long-term deposits (ind_dela_fin_ult1)
Funds (ind_fond_fin_ult1)


THIS ONE MIGHT BE INTERESTING IF WE DON'T FIND ANYTHING BETTER...

# Examples with pension (ind_plan_fin_ult1)

In [229]:
idx_product = product_columns.index('ind_plan_fin_ult1')
print(idx_product)
samples_to_look = search_samples_with_label(dataset, 'train', idx_product, [], [], exclude_same=True)
correct_samples = search_correct_predictions(model, checkpoint_path, dataset, 'train', samples_to_look, k=10, exclude_most_att_last=True)

15
Num samples found: 
179
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
Samples found 1


In [230]:
show_predictions(model, checkpoint_path, dataset, correct_samples[0], 'train')

Index Sample: 133239
ncodpers: 972271
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added Securities (ind_valo_fin_ult1)  -- Attention weight: 0.864279
1: Added Credit Card (ind_tjcr_fin_ult1)  -- Attention weight: 0.135721
Top 7 Predictions: 
Direct Debit (ind_recibo_ult1): 0.213151
Payroll (ind_nomina_ult1): 0.132024
Pensions (ind_nom_pens_ult1): 0.131652
Credit Card (ind_tjcr_fin_ult1): 0.126721
e-account (ind_ecue_fin_ult1): 0.114371
Funds (ind_fond_fin_ult1): 0.111313
Current Accounts (ind_cco_fin_ult1): 0.0975631
Labels: 
Pensions (ind_plan_fin_ult1)


Review co-ocurrence with Securities to check if interesting

# Examples with Current Accounts (ind_cco_fin_ult1)

In [260]:
idx_product = product_columns.index('ind_cco_fin_ult1')
print(idx_product)
samples_to_look = search_samples_with_label(dataset, 'train', idx_product, [], [], exclude_same=True)
correct_samples = search_correct_predictions(model, checkpoint_path, dataset, 'train', samples_to_look, k=1, exclude_most_att_last=True)

2
Num samples found: 2970
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
Samples found 55


In [261]:
show_predictions(model, checkpoint_path, dataset, correct_samples[2], 'train')

Index Sample: 59591
ncodpers: 352085
INFO:tensorflow:Restoring parameters from C:/data/Santander/checkpoints/rnn_attention_embeddings_FinalModel1/best_model/model_best.ckpt-2355712
0: Added e-account (ind_ecue_fin_ult1) Removed Junior Account (ind_ctju_fin_ult1)  -- Attention weight: 0.939558
1: Added Payroll Account (ind_cno_fin_ult1)  -- Attention weight: 0.0604421
Top 7 Predictions: 
Current Accounts (ind_cco_fin_ult1): 0.53313
Payroll Account (ind_cno_fin_ult1): 0.142641
Direct Debit (ind_recibo_ult1): 0.0494539
Pensions (ind_nom_pens_ult1): 0.0178259
Payroll (ind_nomina_ult1): 0.00844065
Credit Card (ind_tjcr_fin_ult1): 0.00661885
Long-term deposits (ind_dela_fin_ult1): 0.00610927
Labels: 
Current Accounts (ind_cco_fin_ult1)
